In [62]:
# importing required packages
from utils import (
    load_json, pd_preview, pd_cols_remove_special_characters, pd_rename_cols, convert_epoch_to_datetime
)
from pandas import json_normalize, notna, read_sql
import sqlite3
from sqlite3 import Error

### Reading the data

In [63]:
#reading brands json data
brands_data = load_json('../data/brands.json')
brands_df = json_normalize(brands_data, sep = '_')

#reading users json data
users_data = load_json('../data/users.json')
users_df = json_normalize(users_data, sep = '_')

#reading receipts json data
receipts_data = load_json('../data/receipts.json')
receipts_df = json_normalize(receipts_data, sep = '_')

### Exploratory Data Analysis before cleaning

In [64]:
brands_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   object
 1   category      1012 non-null   object
 2   categoryCode  517 non-null    object
 3   name          1167 non-null   object
 4   topBrand      555 non-null    object
 5   _id_$oid      1167 non-null   object
 6   cpg_$id_$oid  1167 non-null   object
 7   cpg_$ref      1167 non-null   object
 8   brandCode     933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB


In [65]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   active             495 non-null    bool   
 1   role               495 non-null    object 
 2   signUpSource       447 non-null    object 
 3   state              439 non-null    object 
 4   _id_$oid           495 non-null    object 
 5   createdDate_$date  495 non-null    int64  
 6   lastLogin_$date    433 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.8+ KB


In [66]:
receipts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bonusPointsEarned        544 non-null    float64
 1   bonusPointsEarnedReason  544 non-null    object 
 2   pointsEarned             609 non-null    object 
 3   purchasedItemCount       635 non-null    float64
 4   rewardsReceiptItemList   679 non-null    object 
 5   rewardsReceiptStatus     1119 non-null   object 
 6   totalSpent               684 non-null    object 
 7   userId                   1119 non-null   object 
 8   _id_$oid                 1119 non-null   object 
 9   createDate_$date         1119 non-null   int64  
 10  dateScanned_$date        1119 non-null   int64  
 11  finishedDate_$date       568 non-null    float64
 12  modifyDate_$date         1119 non-null   int64  
 13  pointsAwardedDate_$date  537 non-null    float64
 14  purchaseDate_$date      

In [67]:
print('Receipts')
print('----------------------------')
print(receipts_df.isnull().sum())
print('Users')
print('----------------------------')
print(users_df.isnull().sum())
print('Brands')
print('----------------------------')
print(brands_df.isnull().sum())

Receipts
----------------------------
bonusPointsEarned          575
bonusPointsEarnedReason    575
pointsEarned               510
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
_id_$oid                     0
createDate_$date             0
dateScanned_$date            0
finishedDate_$date         551
modifyDate_$date             0
pointsAwardedDate_$date    582
purchaseDate_$date         448
dtype: int64
Users
----------------------------
active                0
role                  0
signUpSource         48
state                56
_id_$oid              0
createdDate_$date     0
lastLogin_$date      62
dtype: int64
Brands
----------------------------
barcode           0
category        155
categoryCode    650
name              0
topBrand        612
_id_$oid          0
cpg_$id_$oid      0
cpg_$ref          0
brandCode       234
dtype: int64


In [68]:
print(users_df.duplicated().sum())
print(brands_df.duplicated().sum())
print(receipts_df.duplicated(subset=['_id_$oid']).sum())

283
0
0


#### Data cleansing covers Below
1. Renaming columns

2. converting epoch to datetime

3. Removing special characters

4. Splitting receipts into receipts and receipt_items dataframe

In [69]:
# maps of columns to rename
brand_map ={
    '_id_oid' : 'brand_id',
    'cpg_id_oid' : 'cpg_id',
    'categoryCode': 'category_code',
    'topBrand': 'top_brand',
    'name': 'brand_name',
    'brandCode': 'brand_code'
}

users_map ={
    '_id_oid': 'users_id',
    'signUpSource': 'signup_source',
    'createdDate_date': 'created_date',
    'lastLogin_date': 'last_login_date'
}
receipts_map ={
    '_id_oid': 'receipt_id',
    'createDate_date': 'created_date',
    'userId': 'user_id',
    'modifyDate_date': 'modified_date',
    'dateScanned_date': 'date_scanned',
    'finishedDate_date': 'finished_date',
    'pointsAwardedDate_date': 'points_awarded_date',
    'purchaseDate_date': 'purchase_date',
    'totalSpent': 'total_spent',
    'bonusPointsEarned': 'bonus_points_earned',
    'rewardsReceiptStatus': 'rewards_receipt_status',
    'pointsEarned': 'points_earned',
    'purchasedItemCount': 'purchased_item_count'
}

# columns to convert from epoch to datetime
epoch_columns = ['created_date', 'last_login_date',  'modified_date','date_scanned', 'finished_date', 'purchase_date', 'points_awarded_date']

In [70]:
# Applying data cleansing on brands dataframe
brands_clean = pd_cols_remove_special_characters(brands_df,  remove_characters='$', replace_char='')
brands = pd_rename_cols(brands_clean, rename_map=brand_map, ignore_missing=True)



In [71]:
#previewing the dataframe
pd_preview(brands)

,barcode,category,category_code,brand_name,top_brand,brand_id,cpg_id,cpg_ref,brand_code
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...,...
1162,511111116752,Baking,BAKING,test brand @1601644365844,NaN,5f77274dbe37ce6b592e90c0,5f77274dbe37ce6b592e90bf,Cogs,NaN
1163,511111706328,Breakfast & Cereal,NaN,Dippin Dots® Cereal,NaN,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs,DIPPIN DOTS CEREAL
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1598639215217
1165,511111400608,Grocery,NaN,LIPTON TEA Leaves,False,5a021611e4b00efe02b02a57,5332f5f6e4b03c9a25efd0b4,Cogs,LIPTON TEA Leaves


In [72]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   barcode        1167 non-null   object
 1   category       1012 non-null   object
 2   category_code  517 non-null    object
 3   brand_name     1167 non-null   object
 4   top_brand      555 non-null    object
 5   brand_id       1167 non-null   object
 6   cpg_id         1167 non-null   object
 7   cpg_ref        1167 non-null   object
 8   brand_code     933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB


In [73]:
users_clean = pd_cols_remove_special_characters(users_df,  remove_characters='$', replace_char='')
users = pd_rename_cols(users_clean, rename_map=users_map, ignore_missing=True)
for col in users.columns:
    if col in epoch_columns:
        users[col] = users[col].apply(lambda x: convert_epoch_to_datetime(int(x)) if notna(x) else None)

In [74]:
pd_preview(users)

,active,role,signup_source,state,users_id,created_date,last_login_date
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03T15:24:04,2021-01-03T15:25:37
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03T15:24:04,2021-01-03T15:25:37
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03T15:24:04,2021-01-03T15:25:37
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03T15:25:30,2021-01-03T15:25:30
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03T15:24:04,2021-01-03T15:25:37
...,...,...,...,...,...,...,...
490,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19T14:21:22,2021-03-05T16:52:23
491,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19T14:21:22,2021-03-05T16:52:23
492,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19T14:21:22,2021-03-05T16:52:23
493,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19T14:21:22,2021-03-05T16:52:23


In [75]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   active           495 non-null    bool  
 1   role             495 non-null    object
 2   signup_source    447 non-null    object
 3   state            439 non-null    object
 4   users_id         495 non-null    object
 5   created_date     495 non-null    object
 6   last_login_date  433 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB


In [76]:
# Applying the data cleansing steps on receipts df
receipts_df =  pd_cols_remove_special_characters(receipts_df,  remove_characters='$', replace_char='')
receipts = pd_rename_cols(receipts_df, rename_map=receipts_map)
for col in receipts_df.columns:
    if col in epoch_columns:
        receipts[col] = receipts[col].apply(lambda x: convert_epoch_to_datetime(int(x)) if notna(x) else None)

In [77]:
pd_preview(receipts)

,bonus_points_earned,bonusPointsEarnedReason,points_earned,purchased_item_count,rewardsReceiptItemList,rewards_receipt_status,total_spent,user_id,receipt_id,created_date,date_scanned,finished_date,modified_date,points_awarded_date,purchase_date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03T15:25:31,2021-01-03T15:25:31,2021-01-03T15:25:31,2021-01-03T15:25:36,2021-01-03T15:25:31,2021-01-03T00:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03T15:24:43,2021-01-03T15:24:43,2021-01-03T15:24:43,2021-01-03T15:24:48,2021-01-03T15:24:43,2021-01-02T15:24:43
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03T15:25:37,2021-01-03T15:25:37,None,2021-01-03T15:25:42,None,2021-01-03T00:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03T15:25:34,2021-01-03T15:25:34,2021-01-03T15:25:34,2021-01-03T15:25:39,2021-01-03T15:25:34,2021-01-03T00:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03T15:25:06,2021-01-03T15:25:06,2021-01-03T15:25:11,2021-01-03T15:25:11,2021-01-03T15:25:06,2021-01-02T15:25:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603cc0630a720fde100003e6,2021-03-01T10:22:27,2021-03-01T10:22:27,None,2021-03-01T10:22:28,None,2020-08-17T00:00:00
1115,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603d0b710a720fde1000042a,2021-03-01T15:42:41,2021-03-01T15:42:41,None,2021-03-01T15:42:41,None,None
1116,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603cf5290a720fde10000413,2021-03-01T14:07:37,2021-03-01T14:07:37,None,2021-03-01T14:07:37,None,None
1117,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603ce7100a7217c72c000405,2021-03-01T13:07:28,2021-03-01T13:07:28,None,2021-03-01T13:07:29,None,2020-08-17T00:00:00


In [78]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bonus_points_earned      544 non-null    float64
 1   bonusPointsEarnedReason  544 non-null    object 
 2   points_earned            609 non-null    object 
 3   purchased_item_count     635 non-null    float64
 4   rewardsReceiptItemList   679 non-null    object 
 5   rewards_receipt_status   1119 non-null   object 
 6   total_spent              684 non-null    object 
 7   user_id                  1119 non-null   object 
 8   receipt_id               1119 non-null   object 
 9   created_date             1119 non-null   object 
 10  date_scanned             1119 non-null   object 
 11  finished_date            568 non-null    object 
 12  modified_date            1119 non-null   object 
 13  points_awarded_date      537 non-null    object 
 14  purchase_date           

In [79]:
#Moving rewardsReceiptItemList nested list to separate table
if 'rewardsReceiptItemList' in receipts.columns:
    receipts_df_exploded = receipts_df.explode('rewardsReceiptItemList')
    receipts_df_exploded = receipts_df_exploded.dropna(subset=['rewardsReceiptItemList'])
    items_df = json_normalize(receipts_df_exploded['rewardsReceiptItemList'], sep='_')
    items_df.insert(0, 'receipt_id', receipts_df_exploded['receipt_id'].values)
receipt_item_df = items_df

In [80]:
receipt_item_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   receipt_id                          6941 non-null   object 
 1   barcode                             3090 non-null   object 
 2   description                         6560 non-null   object 
 3   finalPrice                          6767 non-null   object 
 4   itemPrice                           6767 non-null   object 
 5   needsFetchReview                    813 non-null    object 
 6   partnerItemId                       6941 non-null   object 
 7   preventTargetGapPoints              358 non-null    object 
 8   quantityPurchased                   6767 non-null   float64
 9   userFlaggedBarcode                  337 non-null    object 
 10  userFlaggedNewItem                  323 non-null    object 
 11  userFlaggedPrice                    299 non

In [81]:
pd_preview(receipt_item_df)

,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,603cc2bc0a720fde100003e9,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6937,603cc0630a720fde100003e6,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6938,603cc0630a720fde100003e6,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6939,603ce7100a7217c72c000405,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


In [82]:
print(users.duplicated().sum())
print(brands.duplicated().sum())
print(receipt_item_df.duplicated().sum())

283
0
0


In [83]:
# count of items scanned for each barcorde
receipt_item_df['brandCode'].value_counts()

brandCode
HY-VEE            291
BEN AND JERRYS    180
PEPSI              93
KROGER             89
KLEENEX            88
                 ... 
PACIFIC FOODS       1
EGGO                1
GRIMMWAY FARMS      1
LA BANDERITA        1
VIVA                1
Name: count, Length: 227, dtype: int64

In [84]:
# count of receipts for each receipt status
receipts['rewards_receipt_status'].value_counts()

rewards_receipt_status
FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: count, dtype: int64

No receipts for accepted status

# 2. Write queries that directly answer predetermined questions from a business stakeholder

### Used Python for examining the quality of data and Sqllite for writing sql queries for answering the below mentioned questions:

- When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

- When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?


In [85]:
db_path ='fetch_rewards.db'

In [86]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [87]:
cursor.executescript("""
DROP TABLE IF EXISTS users_dim;
DROP TABLE IF EXISTS brands_dim;
DROP TABLE IF EXISTS receipts_fact;
DROP TABLE IF EXISTS receipt_items_dim;
CREATE TABLE users_dim (
    user_id TEXT ,
    state TEXT,
    singup_source TEXT,
    created_date DATETIME,
    last_login_date DATETIME,
    role TEXT,
    active BOOLEAN
);

CREATE TABLE brands_dim (
    brand_id TEXT PRIMARY KEY,
    barcode TEXT UNIQUE,
    brand_name TEXT,
    brand_code TEXT,
    category TEXT,
    top_brand BOOLEAN,
    category_code TEXT,
    cpg_id TEXT,
    cpg_ref TEXT               
);

CREATE TABLE receipts_fact (
    receipt_id TEXT PRIMARY KEY,
    user_id TEXT,
    purchase_date DATETIME,
    total_spent FLOAT,
    bonus_points_earned FLOAT,
    date_scanned DATETIME,
    purchased_item_count FLOAT,
    created_date DATETIME,
    points_awarded_date DATETIME,
    bonusPointsEarnedReason TEXT,              
    points_earned FLOAT,
    modified_date DATETIME,
    finished_date DATETIME,
    rewards_receipt_status TEXT,
    FOREIGN KEY(user_id) REFERENCES users_dim(user_id)
);

CREATE TABLE receipt_items_dim (
    item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    receipt_id TEXT,
    barcode TEXT,
    description TEXT,
    finalPrice FLOAT,
    itemPrice FLOAT,
    needsFetchReview BOOLEAN,
    partnerItemId TEXT,
    preventTargetGapPoints BOOLEAN,
    quantityPurchased FLOAT,
    userFlaggedBarcode TEXT,
    userFlaggedNewItem BOOLEAN,
    userFlaggedPrice FLOAT,
    userFlaggedQuantity FLOAT,
    needsFetchReviewReason TEXT,
    pointsNotAwardedReason TEXT,
    pointsPayerId TEXT,
    rewardsGroup TEXT,
    rewardsProductPartnerId TEXT,
    userFlaggedDescription TEXT,
    originalMetaBriteBarcode TEXT,
    originalMetaBriteDescription TEXT,
    brandCode TEXT,
    competitorRewardsGroup TEXT,
    discountedItemPrice  FLOAT,
    originalReceiptItemText TEXT,
    itemNumber TEXT,
    originalMetaBriteQuantityPurchased INTEGER,
    pointsEarned  FLOAT,
    targetPrice FLOAT,
    competitiveProduct BOOLEAN,
    originalFinalPrice FLOAT,
    originalMetaBriteItemPrice FLOAT,
    deleted BOOLEAN,
    priceAfterCoupon FLOAT,
    metabriteCampaignId TEXT,
    FOREIGN KEY(receipt_id) REFERENCES receipts_fact(receipt_id),
    FOREIGN KEY(barcode) REFERENCES brands_dim(barcode)
);
                                 
""")

In [88]:
conn.commit()

In [89]:
users.to_sql("users_dim", conn, if_exists="replace", index=False)

495

In [90]:
brands.to_sql("brands_dim", conn, if_exists="replace", index=False)

1167

In [91]:
receipts_fact = receipts.drop(['rewardsReceiptItemList'], axis=1)

In [92]:
receipts_fact.to_sql("receipts_fact", conn, if_exists="replace", index=False)

1119

In [93]:
receipt_item_df.to_sql("receipt_items_dim", conn, if_exists="replace", index=False)

6941

#### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [94]:
query_avg_spend ="""
        SELECT 
            rewards_receipt_status,
            AVG(total_spent) AS avg_spend
        FROM receipts_fact
        WHERE rewards_receipt_status IN ('ACCEPTED', 'REJECTED', 'FINISHED')
        GROUP BY rewards_receipt_status
"""

In [95]:
avg_spend_receipts = read_sql(query_avg_spend, conn)

In [96]:
avg_spend_receipts

,rewards_receipt_status,avg_spend
0,FINISHED,80.854305
1,REJECTED,23.326056


##### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [97]:
query_total_items_purchased ="""
    SELECT 
        r.rewards_receipt_status,
        SUM(r.purchased_item_count ) AS total_items
    FROM receipts_fact r
    WHERE r.rewards_receipt_status IN ('ACCEPTED', 'REJECTED', 'FINISHED')
    GROUP BY r.rewards_receipt_status
"""

In [98]:
total_items_purchased = read_sql(query_total_items_purchased, conn)
total_items_purchased

,rewards_receipt_status,total_items
0,FINISHED,8184.0
1,REJECTED,173.0


# 3. DATA QUALITY

## **1. Missing Value Analysis**

### **Receipts Dataset**
The following columns contain missing values in the `receipts` dataset:

| Column                     | Missing Values | Percentage Missing |
|----------------------------|---------------|--------------------|
| `bonusPointsEarned`        | 575           | High Missing Rate |
| `bonusPointsEarnedReason`  | 575           | High Missing Rate |
| `pointsEarned`             | 510           | Significant Missing Data |
| `purchasedItemCount`       | 484           | Data Incompleteness |
| `rewardsReceiptItemList`   | 440           | Missing Item Details |
| `totalSpent`               | 435           | Important for financial analysis |
| `finishedDate_$date`       | 551           | Transaction completion date missing |
| `pointsAwardedDate_$date`  | 582           | Reward processing delays |
| `purchaseDate_$date`       | 448           | Purchase records may be incomplete |

#### **Observations:**
- `bonusPointsEarned`, `bonusPointsEarnedReason`, `pointsEarned`, `purchasedItemCount`, and `totalSpent` have a significant number of missing values.
- `purchaseDate` is missing in **40%** of the records.

---

### **Users Dataset**
The following columns contain missing values in the `users` dataset:

| Column          | Missing Values | Percentage Missing |
|----------------|---------------|--------------------|
| `signUpSource` | 48            | Registration origin unknown |
| `state`        | 56            | Location missing for users |
| `lastLogin_$date` | 62        | Activity tracking gaps |

#### **Observations:**
- `lastLogin` is missing in **13%** of the users.
- `state` and `signUpSource` have missing values.

---

### **Brands Dataset**
The following columns contain missing values in the `brands` dataset:

| Column          | Missing Values | Percentage Missing |
|----------------|---------------|--------------------|
| `category`     | 155           | Affects product classification |
| `categoryCode` | 650           | High missing rate |
| `topBrand`     | 612           | Brand ranking incomplete |
| `brandCode`    | 234           | Missing brand identifier |

#### **Observations:**
- `categoryCode` is missing in **56%** of the rows.
- `topBrand` has many missing values (**53% missing**).
- `brandCode` is missing for **20%** of rows.

---

## **2. Duplicate Records**
- The `_id` column in `users_df` is **not unique**, which indicates duplicate user records.

---

## **3. Inconsistent Data Types**
### **brands_df:**
- `barcode` is stored as **integer**, which should be **string** for consistency.

### **receipts_df:**
- `createDate`, `purchaseDate`, and `modifyDate` are stored as **strings**, but they should be **datetime** for proper analysis.

---

## **4. Outliers in Numerical Columns**
- `totalSpent` in `receipts_df` likely contains **outliers**, as some receipts might have unrealistically high values.

---

## **5. Unexpected Categorical Values**
- `rewardsReceiptStatus` in `receipts_df` should have predefined values (**e.g., ACCEPTED, REJECTED, FINISHED**).
- Need to check for **unexpected values**.

---

## **6. Date Consistency Issues**
- `purchaseDate` in `receipts_df` should not be **future dates** or **before the user's `createdDate`**.
- `lastLogin` in `users_df` should not be **before `createdDate`**.

---

## **7. Referential Integrity Issues**
- Some `userId` values in `receipts_df` might **not exist** in `users_df`, indicating **orphaned records**.

### **Steps should be taken after discussing with business or stakeholders:**
- **Handle missing values** by imputing or removing them based on business logic.
- **Fix duplicates** by keeping the first occurrence or merging records.
- **Ensure consistent data types** before further analysis.
- **Detect and remove outliers** that might skew results.
- **Validate categorical fields** and restrict values to expected ones.
- **Fix date inconsistencies** to prevent incorrect reporting.
- **Enforce referential integrity** to maintain accurate relationships.

## ** END OF ANALYSIS **